There are a number of ways to register tools with an agent:
- via the @agent.tool decorator - for tools that need access to the agent context
- via the @agent.tool_plan decorator - for tools that do not need access to the agent context
- via the tools keyword argument to Agent which can take either plain functions or instances of Tool

In [17]:
import random
import asyncio
from pprint import pprint
from pydantic_ai import Agent, RunContext

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [15]:


agent = Agent(
    'google-gla:gemini-1.5-flash',  
    deps_type=str,  
    system_prompt=(
        "You're a dice game, you should roll the die and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Use the player's name in the response."
    ),
)


@agent.tool_plain  
def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


@agent.tool  
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


dice_result = agent.run_sync('My guess is 4', deps='Anne')  
print(dice_result.data)

Anne, you rolled a 5.  Better luck next time!



In [18]:
pprint(dice_result.all_messages())

[ModelRequest(parts=[SystemPromptPart(content="You're a dice game, you should "
                                              'roll the die and see if the '
                                              'number you get back matches the '
                                              "user's guess. If so, tell them "
                                              "they're a winner. Use the "
                                              "player's name in the response.",
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='My guess is 4',
                                    timestamp=datetime.datetime(2025, 2, 11, 3, 3, 55, 484513, tzinfo=datetime.timezone.utc),
                                    part_kind='user-prompt')],
              kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_player_name',
                                   args={},
                